In [517]:

# # Example data
# x_PLOT = 1:15
# y1_PLOT = []  # Random data for the first graph
# y2_PLOT = []  # Random data for the second graph


In [531]:
using ITensors, ITensorMPS
using LinearAlgebra
using Plots
using HDF5

N = 5

f4 = h5open("Basis_states/MPO_$N.h5","r")
W1 = read(f4,"W",MPO)
close(f4)
println(maxlinkdim(W1))

s = siteinds(W1)


4


5-element Vector{Vector{Index{Int64}}}:
 [(dim=2|id=502|"Qubit,Site,n=1")', (dim=2|id=502|"Qubit,Site,n=1")]
 [(dim=2|id=689|"Qubit,Site,n=2")', (dim=2|id=689|"Qubit,Site,n=2")]
 [(dim=2|id=453|"Qubit,Site,n=3")', (dim=2|id=453|"Qubit,Site,n=3")]
 [(dim=2|id=475|"Qubit,Site,n=4")', (dim=2|id=475|"Qubit,Site,n=4")]
 [(dim=2|id=324|"Qubit,Site,n=5")', (dim=2|id=324|"Qubit,Site,n=5")]

In [532]:
# Taking the input 

x = range(0, stop=2π, length=2^N)
input_array = cos.(x)
    
#ψ = MPS(input_array,s;cutoff=cutoff,maxdim=maxdim)
using HDF5
MPS1 = []
for i in 1:2^N
    f1 = h5open("Basis_states/MPS_$N/MPS_create_$i.h5","r")
    mps1 = read(f1,"M",MPS)
    close(f1)
    push!(MPS1,mps1)
end

array = input_array / norm(input_array) # Input

ψ = array[1] *MPS1[1]
for i in 2:2^N
    
    ψ += array[i] * MPS1[i]
end

println(maxlinkdim(ψ))

3


In [533]:
orthogonalize!(ψ, 1) # Orthogonalize Psi at the first index



start = time()
result = contract(W1, ψ)
end_time = time()


1.731577526434942e9

In [534]:
inner_products = []

MPS2 = []
for i in 1:2^N
    f2 = h5open("Basis_states/MPS_$N/MPS_measure_$i.h5","r")
    mps1 = read(f2,"M1",MPS)
    push!(MPS2,mps1)
    close(f2)
end

for i in 1:(2^N)
    push!(inner_products, inner(MPS2[i], result))
    print(inner(MPS2[i], result)*norm(input_array)*2^(N/2))
    println()
end

time1 = end_time - start

1.0000000000000209 + 1.8272640836402662e-16im
16.149735723719303 - 1.59061013527649im
-0.35383322373041654 + 0.07038180418977555im
-0.12660477936251852 + 0.03840513994846441im
-0.06413657357668608 + 0.026566238619565356im
-0.03755382107950081 + 0.02007293556453266im
-0.02371096309372764 + 0.015843159023778374im
-0.015593022394804533 + 0.012796862764310355im
-0.010448919366975111 + 0.010448919367167771im
-0.007013563834227061 + 0.008546052258932272im
-0.004638225791299282 + 0.006941595447363017im
-0.0029631627927856714 + 0.005543687668022654im
-0.0017773659607031261 + 0.0042909410076889766im
-0.000952464877768829 + 0.0031398559115437064im
-0.0004093156156598836 + 0.002057768559051133im
-0.0001003222427091 + 0.0010185888240781316im
3.927164442693566e-14 - 1.6394410339679017e-13im
-0.00010032224270592084 - 0.0010185888240742647im
-0.0004093156157495892 - 0.002057768559528139im
-0.0009524648777755952 - 0.0031398559116154827im
-0.0017773659605493481 - 0.004290941006841886im
-0.0029631627927

0.004105091094970703

In [535]:
using FFTW

start2 = time()
x = fft(input_array)
end2 = time()

for a in x
    println(a)
    println()
end

time2 = end2 - start2


0.9999999999999998 + 0.0im

16.149735723719495 + 1.5906101352764617im

-0.3538332237303783 - 0.0703818041897859im

-0.12660477936253065 - 0.03840513994846283im

-0.06413657357673683 - 0.02656623861962376im

-0.037553821079491075 - 0.020072935564491586im

-0.023710963093742023 - 0.01584315902373093im

-0.015593022394802036 - 0.012796862764329842im

-0.01044891936704373 - 0.010448919367044396im

-0.007013563834202541 - 0.008546052258911974im

-0.004638225791212473 - 0.006941595447672394im

-0.0029631627927938983 - 0.0055436876680278815im

-0.001777365960343967 - 0.004290941006761917im

-0.0009524648777894662 - 0.003139855911602138im

-0.0004093156157039257 - 0.002057768559572855im

-0.00010032224272293888 - 0.001018588824037936im

2.220446049250313e-16 + 0.0im

-0.00010032224272293888 + 0.001018588824037936im

-0.0004093156157039257 + 0.002057768559572855im

-0.0009524648777894662 + 0.003139855911602138im

-0.001777365960343967 + 0.004290941006761917im

-0.0029631627927938983 + 0.0055436

0.00057220458984375

In [536]:
println("Time taken QFT: ", time1, " seconds")
println("Time taken FFT: ", time2, " seconds")
push!(y1_PLOT, time1)
push!(y2_PLOT, time2)


Time taken QFT: 0.004105091094970703 seconds
Time taken FFT: 0.00057220458984375 seconds


17-element Vector{Any}:
 0.002853870391845703
 0.0003540515899658203
 0.004642963409423828
 0.00030612945556640625
 0.0005178451538085938
 0.0017349720001220703
 0.0002830028533935547
 0.0002429485321044922
 0.000286102294921875
 0.00024700164794921875
 0.0002799034118652344
 0.0006561279296875
 0.0011670589447021484
 0.0004420280456542969
 0.0006799697875976562
 0.0003230571746826172
 0.00057220458984375

In [524]:
# using Plots

# # Create the plot with pythonplot backend
# p = plot(
#     x_PLOT, y1_PLOT,
#     label = "QFT",
#     linewidth = 2,
#     xticks = 1:1:length(x_PLOT)
# )
# plot!(
#     p, x_PLOT, y2_PLOT,
#     label = "FFT",
#     linewidth = 2
# )

# # Add axis labels and title
# xlabel!("X Axis")
# ylabel!("Y Axis")
# title!("Comparison of Two Lines")

# # Display the plot
# display(p)